# GenAI HW6: LLM Values Alignment
## Objectives
- Learn how to align a model's behavior using labelled preference data.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to ntu-gen-ai-2024-spring-ta@googlegroups.com

## Install and import necessary libraries  (~2 min)
### Ignore the warning if the blockes finish successfully.

In [1]:
!pip install -q bitsandbytes==0.43.1 datasets==2.19.0 peft==0.10.0 trl==0.8.6 accelerate==0.29.3

^C
ERROR: Operation cancelled by user


## Load dataset

In [4]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

fatal: 目标路径 'GenAI_hw6_dataset' 已经存在，并且不是一个空目录。


## Import Packages

In [1]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer

/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Open and load the json dataset
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

## Load model

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'MediaTek-Research/Breeze-7B-Instruct-v0_1',
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.52s/it]


## Get response from the original model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output)
    print('Response from original model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
 10%|█         | 1/10 [00:03<00:30,  3.41s/it]

Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


 20%|██        | 2/10 [00:06<00:24,  3.11s/it]

...

Response from original model:
真人化可能促進日本漫畫的創新和多樣性，因真人版電影或劇集可以吸引新觀眾群，並提供不同的故事表現方式。然而，過度的真人化或商業化可能限制創意和多元性。平衡點須考慮。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [00:26<00:02,  2.96s/it]

Response from original model:
真人化可能有助於擴大動漫文化的市場份額，因它能吸引更多非動漫迷觀眾，並提高大眾對動漫文化的認識與接受度。然而，成功與否仍取決於作品品質、行銷策略和市場接受程度。

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [00:28<00:00,  2.90s/it]

Response from original model:
真人化可能有助於提高日本漫畫在全球的競爭力，因它能吸引更多觀眾並拓展市場，但同時也可能限制原創性，影響日本漫畫的獨特性。



## Set parameters
### You only need to modify this block. Please don’t alter any other parts.

In [5]:
num_epoch = 1  # 1
data_size = 50
support_ratio = 1

## Prepare training data

In [6]:
# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],support,真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],support,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],support,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],support,真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],support,真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],support,真人化可以透過場景、服裝等元素保留文化特色。,真人化可能因為文化差異而失去原作獨有的文化魅力。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],support,真人化促使技術創新，挑戰視覺效果上的新高度。,真人化可能因為技術限制而無法達到動畫中的視覺效果。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],support,演員選擇可因應市場需求，不必受限於動畫形象。,演員選擇可能引起爭議，觀眾難以接受角色塑造。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],support,真人化有機會讓更多人接受動漫，提高社會認同度。,真人化可能因為劇情改編而無法贏得社會認同。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],support,真人化可以透過演員表現保留角色的個性特色。,真人化可能因演員演技或導演選擇而失去角色的原有特色。


## Training

In [7]:
training_args = TrainingArguments(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

In [8]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/home/scy/anaconda3/envs/llm/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 50/50 [00:00<00:00, 1757.41 examples/s]


In [10]:
dpo_trainer.train()

 17%|█▋        | 1/6 [00:03<00:18,  3.66s/it]

{'loss': 0.6931, 'grad_norm': 3.312622547149658, 'learning_rate': 0.0002, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -49.23818588256836, 'logps/chosen': -42.10595703125, 'logits/rejected': -0.332323282957077, 'logits/chosen': -0.32404395937919617, 'epoch': 0.16}


 33%|███▎      | 2/6 [00:07<00:14,  3.62s/it]

{'loss': 0.6931, 'grad_norm': 3.633286952972412, 'learning_rate': 0.00016, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/rejected': -46.570377349853516, 'logps/chosen': -42.9771842956543, 'logits/rejected': -0.21861951053142548, 'logits/chosen': -0.30097079277038574, 'epoch': 0.32}


 50%|█████     | 3/6 [00:10<00:10,  3.58s/it]

...

100%|██████████| 6/6 [00:21<00:00,  3.61s/it]

{'loss': 0.2922, 'grad_norm': 2.238243818283081, 'learning_rate': 0.0, 'rewards/chosen': -0.20596656203269958, 'rewards/rejected': -1.522858738899231, 'rewards/accuracies': 0.875, 'rewards/margins': 1.3168922662734985, 'logps/rejected': -68.4411392211914, 'logps/chosen': -48.29946517944336, 'logits/rejected': -0.21081894636154175, 'logits/chosen': -0.23252210021018982, 'epoch': 0.96}


100%|██████████| 6/6 [00:22<00:00,  3.82s/it]

{'train_runtime': 22.9156, 'train_samples_per_second': 2.182, 'train_steps_per_second': 0.262, 'train_loss': 0.47796518603960675, 'epoch': 0.96}


TrainOutput(global_step=6, training_loss=0.47796518603960675, metrics={'train_runtime': 22.9156, 'train_samples_per_second': 2.182, 'train_steps_per_second': 0.262, 'total_flos': 0.0, 'train_loss': 0.47796518603960675, 'epoch': 0.96})

## Get response from the trained model

In [11]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f'Question {id}:\n'+data['prompt'])
    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config=GenerationConfig(
            do_sample=False,
            max_new_tokens = 200,
            pad_token_id = tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output)
    print('Response from trained model:\n'+output+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？


 10%|█         | 1/10 [00:01<00:12,  1.41s/it]

Response from trained model:
真人化可以提高日本漫畫的全球可及性，吸引更多觀眾並拓展市場。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？


 20%|██        | 2/10 [00:03<00:14,  1.82s/it]

...

Response from trained model:
真人化可以促進日本漫畫的創新和多樣性，因它能吸引更多觀眾、拓展市場、並提供不同形式的創作。然而，平衡商業利益與藝術表現是關鍵。

Question 9:
真人化是否有助於擴大動漫文化的市場份額？


 90%|█████████ | 9/10 [00:20<00:02,  2.41s/it]

Response from trained model:
真人化有助於擴大動漫文化的市場份額，因它能吸引更多非動漫迷觀眾，拓展受眾群，並促進跨文化交流。

Question 10:
真人化是否有助於提高日本漫畫在全球的競爭力？


100%|██████████| 10/10 [00:22<00:00,  2.21s/it]

Response from trained model:
真人化可以提高日本漫畫在全球的競爭力，因它能吸引更多觀眾、拓展市場、增加收入和影響力。



## Please observe the output of this block to complete your report, and don't forget to take a screenshot of the results

In [12]:
model_response = []
print(f'num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}')
print()
for data in test_data:
    id = data['id']
    ref_output = original_model_response[id-1]
    output = trained_model_response[id-1]
    print(f'Question {id}:\n'+data['prompt'])
    print('Response from original model:\n'+ref_output)
    print('Response from trained model:\n'+output)
    print()
    model_response.append({'id':data['id'], 'prompt':data['prompt'], 'response_from_original_model':ref_output, 'response_from_trained_model':output})

num_epoch: 1
data_size: 50
support_ratio: 1

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。
Response from trained model:
真人化可以提高日本漫畫的全球可及性，吸引更多觀眾並拓展市場。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注和支持日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。
Response from trained model:
真人化影響年輕一代對日本漫畫的看法，使之更容易理解故事、角色性格，並吸引更多觀眾，促進文化交流。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。
Response from trained model:
真人化可以提高原作漫畫的能見度和受歡迎程度，但文學價值主要取決於原作者的故事和文字品質，真人化只是一種呈現方式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。
Response from trained model:
真人化有助於保護和保存日本漫畫的傳統，因它能吸引更多觀眾、拓展市場、並傳承故事與藝術形式。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以

## Get the output file

In [13]:
with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
    json.dump(model_response, outfile, indent=4, ensure_ascii=False)